In [38]:
import csv
import jieba
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
import requests
from nltk.tokenize import WordPunctTokenizer
import jieba
import random
from bs4 import BeautifulSoup
import unicodedata
import json
from nltk.tokenize import WordPunctTokenizer
import numpy as np

In [2]:
#去除非中文
def is_chinese(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False

def is_number(uchar):
    """判断一个unicode是否是数字"""
    if uchar >= u'\u0030' and uchar <= u'\u0039':
        return True
    else:
        return False

def is_alphabet(uchar):
    """判断一个unicode是否是英文字母"""
    if (uchar >= u'\u0041' and uchar <= u'\u005a') or (uchar >= u'\u0061' and uchar <= u'\u007a'):
        return True
    else:
        return False
def format_str(content):
    # content = unicode(content,'utf-8')
    content = str(content)
    content_str = ''

    for i in content:
        if is_chinese(i):
            content_str = content_str+i
    return content_str


In [3]:
def userAgent():
    user_agents = [
        "Mozilla/5.0 (Linux; U; Android 2.3.6; en-us; Nexus S Build/GRK39F) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "Avant Browser/1.2.789rel1 (http://www.avantbrowser.com)",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
        "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.27 (KHTML, like Gecko) Chrome/12.0.712.0 Safari/534.27",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.24 Safari/535.1",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.874.120 Safari/535.2",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0 x64; en-US; rv:1.9pre) Gecko/2008072421 Minefield/3.0.2pre",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.10) Gecko/2009042316 Firefox/3.0.10",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-GB; rv:1.9.0.11) Gecko/2009060215 Firefox/3.0.11 (.NET CLR 3.5.30729)",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6 GTB5",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; tr; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 ( .NET CLR 3.5.30729; .NET4.0E)",
        "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
        "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
        "Mozilla/5.0 (Windows NT 5.1; rv:5.0) Gecko/20100101 Firefox/5.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0a2) Gecko/20110622 Firefox/6.0a2",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:7.0.1) Gecko/20100101 Firefox/7.0.1",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b4pre) Gecko/20100815 Minefield/4.0b4pre",
        "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT 5.0 )", ]

    UserAgent = random.choice(user_agents)
    return UserAgent


In [4]:
import random
def get_ip_list(url, headers):
  web_data = requests.get(url, headers=headers)
  soup = BeautifulSoup(web_data.text, 'lxml')
  ips = soup.find_all('tr')
  ip_list = []
  for i in range(1, len(ips)):
    ip_info = ips[i]
    tds = ip_info.find_all('td')
    ip_list.append(tds[1].text + ':' + tds[2].text)
  return ip_list
def get_random_ip(ip_list):
  proxy_list = []
  for ip in ip_list:
    proxy_list.append('http://' + ip)
  proxy_ip = random.choice(proxy_list)
  proxies = {'http': proxy_ip}
  return proxies

In [5]:
##对每个title 分词
def JieBaTitle (titleName):
    finTitle = []
    for index in range(len(titleName)):
        thisTitle = []

        titl = ''

        titl = titl.join((titleName[index]).replace(',', '').replace(': ', ''))
        titlez = jieba.cut_for_search(titl)
        for item in titlez:
            thisTitle.append(item)
        finTitle.append(thisTitle)
    return finTitle

In [299]:
fnews=pd.read_csv('D:\\Soong\\Soong\\dongfangcaifu\\sancsv\\dongfang0.csv')

# fnews2=pd.read_csv('D:\\Soong\\Soong\\dongfangcaifu\\total2.csv')
# fnews3=pd.read_csv('D:\\Soong\\Soong\\dongfangcaifu\\total3.csv')

In [301]:
len(fnews)

817

In [12]:
fnews1=pd.read_csv('D:\\Soong\\Soong\\dongfangcaifu\\total1.csv')

In [13]:
finTitle =JieBaTitle(fnews['title'])

# finTitle2 =JieBaTitle(fnews2['title'])
# finTitle3 =JieBaTitle(fnews3['title'])

In [14]:
finTitle1 =JieBaTitle(fnews1['title'])

In [337]:
##得到每个title对应的名称

def getCompanyNameOfTitle(finTitle):
    companys = []
    totalcompanys = []

    for index in range(len(finTitle)):
        #     for index in range(10):
        total = []  # 保存所有的公司，并且防止出现重复
        time.sleep(1)
        for indexT in range(len(finTitle[index])):  # 对每一个标题
            time.sleep(3)
            if (len(finTitle[index][indexT]) > 1):
                proxy_list = [
                    {"http" : "124.88.67.81:80"},
                    {"http" : "124.88.67.81:80"},
                    {"http" : "124.88.67.81:80"},
                    {"http" : "124.88.67.81:80"},
                    {"http" : "124.88.67.81:80"}
                ]
                proxy = random.choice(proxy_list)
                url = 'http://ipo.qichacha.com/search?key=' + finTitle[index][indexT]
                agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
                cookie = 'qgqp_b_id=de6f5b2aab0bf1b85367cff30462fbd1; st_pvi=64069861991421; st_sp=2019-01-24%2018%3A36%3A57; st_si=94918195918945; st_asi=delete; bdshare_firstime=1548326400263; st_sn=12; st_psi=2019012418445425-111000300841-9825945936'
                refer = 'http://finance.eastmoney.com/news/cgnjj_1.html'
                headers = {'User-Agent': userAgent(), 'Cookie': cookie, 'Refer': refer, 'proxies':proxy}

                r = requests.get(url, headers)
                r.encoding = 'utf-8'
                soup = BeautifulSoup(r.text, 'html.parser')
                time.sleep(1)
                a = 0
                for eachurl in soup.select('tbody'):

                    a = a + 1

                    if (a == 1):
                        hrefCom = eachurl.select('a')[0]
                        hrefCom = format_str(hrefCom)
                        if hrefCom not in total:
                            total.append(hrefCom)

                    # title = soup.select('tbody > a')[0]
        #                     print(hrefCom)
        # companys.append(hrefCom)
        totalcompanys.append(total)

    realTotCom = []
    for index in range(len(totalcompanys)):
        realFinCom = []
        for indexG in range(len(totalcompanys[index])):
            if totalcompanys[index][indexG] in fnews['title'][index]:
                realFinCom.append(totalcompanys[index][indexG])

        realTotCom.append(realFinCom)
    return realTotCom

In [9]:
realTotCom=getCompanyNameOfTitle(finTitle)


In [ ]:
realTotCom1=getCompanyNameOfTitle(finTitle1)



In [ ]:
realTotCom2=getCompanyNameOfTitle(finTitle2)
realTotCom3=getCompanyNameOfTitle(finTitle3)



In [272]:
realTotCom

[[],
 [],
 ['喜临门'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['集团'],
 [],
 [],
 ['金自天正'],
 [],
 [],
 [],
 [],
 [],
 ['正川股份'],
 [],
 [],
 [],
 [],
 [],
 ['天顺股份'],
 [],
 ['集成'],
 [],
 ['佛燃股份'],
 [],
 ['中宠股份'],
 [],
 ['太极股份'],
 [],
 ['永鼎股份'],
 [],
 [],
 [],
 ['博云新材'],
 [],
 ['大连港'],
 [],
 ['浙江美大'],
 [],
 ['大连圣亚'],
 [],
 ['美的集团'],
 [],
 ['泰尔股份'],
 [],
 ['奥康国际'],
 [],
 ['正源'],
 [],
 ['秋林'],
 [],
 [],
 [],
 ['苏盐井神', '汇鸿集团'],
 [],
 ['川润股份'],
 [],
 ['邦宝益智'],
 [],
 [],
 [],
 ['宏创控股'],
 [],
 ['金字火腿'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['漳泽电力'],
 [],
 ['安琪酵母'],
 [],
 ['沧州明珠'],
 [],
 ['天津港'],
 [],
 ['香溢融通'],
 [],
 ['国海证券'],
 [],
 [],
 [],
 ['王府井'],
 [],
 ['南兴装备'],
 [],
 ['以岭药业'],
 [],
 ['百大集团'],
 [],
 ['首商股份'],
 [],
 ['湖南投资'],
 [],
 [],
 [],
 ['山东墨龙'],
 [],
 ['上柴股份'],
 [],
 ['凡谷'],
 [],
 ['思源电气'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['海马汽车'],
 [],
 [],
 [],
 ['全通教育'],
 [],
 [],
 ['全聚德'],
 [],
 [],
 ['恒顺醋业', '伊利股份'],
 [],
 ['椰岛'],
 ['雏鹰农牧'],
 [],
 ['佳沃股份'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],

In [ ]:
#4/14开始运行

In [302]:
fnews['realTotCom']=realTotCom
# fnews['realTotCom1']=realTotCom1
# fnews['realTotCom2']=realTotCom2
# fnews['realTotCom3']=realTotCom3

In [281]:
##返回去在检测一遍
#testCom 最终的公司名称
def FinalGetCompanyNameOfTitle(realTotCom):
    testCom = []
    for index in range(len(realTotCom)):
        testC = []
        for indexT in range(len(realTotCom[index])):
            if (realTotCom[index][indexT] in fnews['title'][index]):
                testC.append(realTotCom[index][indexT])
                break  #先看一个的
        testCom.append(testC)

    return testCom

In [282]:
CompNameArr=FinalGetCompanyNameOfTitle(realTotCom)

In [303]:
fnews['realTotCom']=CompNameArr

In [304]:
fnews.head()

,title,date,source,content,realTotCom
0,科创板亏损第一股年亏26亿背后：核心技术依赖母公司 与台积电相差3代,2019年03月24日 00:07,读懂新三板<span>来源：</span>,NaN,[]
1,读懂新三板,[<p> 芯片产业国产化迫在眉睫！ 2018 年，我国集成电路行业实现销售收入2519....,NaN,NaN,[]
2,全年净利润预亏超4亿 “床垫第一股”喜临门豪掷重金跨界受阻,2019年03月24日 09:29,中国经营网<span>来源：</span>,NaN,[喜临门]
3,中国经营网,"[<p style=""text-align:center;""><a class=""EmIma...",NaN,NaN,[]
4,滨江服务超七成业务来源母公司 戚金兴资本版图再谋扩张,2019年03月24日 09:23,中国经营网<span>来源：</span>,NaN,[]


In [287]:
# fnews.dropna(subset=['content'], inplace = True)

In [305]:
fnews.head()

,title,date,source,content,realTotCom
0,科创板亏损第一股年亏26亿背后：核心技术依赖母公司 与台积电相差3代,2019年03月24日 00:07,读懂新三板<span>来源：</span>,NaN,[]
1,读懂新三板,[<p> 芯片产业国产化迫在眉睫！ 2018 年，我国集成电路行业实现销售收入2519....,NaN,NaN,[]
2,全年净利润预亏超4亿 “床垫第一股”喜临门豪掷重金跨界受阻,2019年03月24日 09:29,中国经营网<span>来源：</span>,NaN,[喜临门]
3,中国经营网,"[<p style=""text-align:center;""><a class=""EmIma...",NaN,NaN,[]
4,滨江服务超七成业务来源母公司 戚金兴资本版图再谋扩张,2019年03月24日 09:23,中国经营网<span>来源：</span>,NaN,[]


In [306]:
np_data = np.array(fnews)
pd_data = pd.DataFrame(np_data,columns=['title','date','source' ,'content','realTotCom'])

In [307]:
pd_data.to_csv('D:\\Soong\\Soong\\dongfangcaifu\\dongfangcaifuCompany.csv')

In [308]:
fnews=pd.read_csv('D:\\Soong\\Soong\\dongfangcaifu\\dongfangcaifuCompany.csv')
fnews.head()

,Unnamed: 0,title,date,source,content,realTotCom
0,0,科创板亏损第一股年亏26亿背后：核心技术依赖母公司 与台积电相差3代,2019年03月24日 00:07,读懂新三板<span>来源：</span>,NaN,[]
1,1,读懂新三板,[<p> 芯片产业国产化迫在眉睫！ 2018 年，我国集成电路行业实现销售收入2519....,NaN,NaN,[]
2,2,全年净利润预亏超4亿 “床垫第一股”喜临门豪掷重金跨界受阻,2019年03月24日 09:29,中国经营网<span>来源：</span>,NaN,['喜临门']
3,3,中国经营网,"[<p style=""text-align:center;""><a class=""EmIma...",NaN,NaN,[]
4,4,滨江服务超七成业务来源母公司 戚金兴资本版图再谋扩张,2019年03月24日 09:23,中国经营网<span>来源：</span>,NaN,[]


In [309]:
# for index in range(len(fnews['realTotCom'])):
#     if not fnews['realTotCom'][index].strip():
#         fnews['realTotCom'][index]='NaN'
testCom=fnews['realTotCom']
testCom

0            []
1            []
2       ['喜临门']
3            []
4            []
5            []
6            []
7            []
8            []
9            []
10           []
11           []
12       ['集团']
13           []
14           []
15     ['金自天正']
16           []
17           []
18           []
19           []
20           []
21     ['正川股份']
22           []
23           []
24           []
25           []
26           []
27     ['天顺股份']
28           []
29       ['集成']
         ...   
787          []
788          []
789    ['华电重工']
790          []
791          []
792          []
793          []
794          []
795          []
796          []
797    ['荣晟环保']
798          []
799          []
800          []
801    ['药明康德']
802          []
803          []
804          []
805          []
806    ['温氏股份']
807          []
808          []
809          []
810          []
811    ['宁德时代']
812          []
813          []
814          []
815          []
816          []
Name: realTotCom, Length

In [293]:
def getCodeOfCompany(testCom):
    # 公式代码:testCode
    testCode = []
    for index in range(len(testCom)):
        testCo = []
        for indexT in range(len(testCom[index])):
            if len(testCom[index][indexT]) > 0:
                url = 'http://ipo.qichacha.com/search?key=' + testCom[index][indexT]
                agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
                cookie = 'qgqp_b_id=de6f5b2aab0bf1b85367cff30462fbd1; st_pvi=64069861991421; st_sp=2019-01-24%2018%3A36%3A57; st_si=94918195918945; st_asi=delete; bdshare_firstime=1548326400263; st_sn=12; st_psi=2019012418445425-111000300841-9825945936'
                refer = 'http://finance.eastmoney.com/news/cgnjj_1.html'
                headers = {'User-Agent': userAgent(), 'Cookie': cookie, 'Refer': refer}

                r = requests.get(url, headers)
                r.encoding = 'utf-8'
                soup = BeautifulSoup(r.text, 'html.parser')

                a = 0
                for eachurl in soup.select('tbody'):

                    a = a + 1

                    if (a == 1):
                        hrefCom = eachurl.select('span')[0]
                        hrefCom = str(hrefCom).replace('<span style="font-size:14px;">', '').replace('</span>', '')
                        #                             print(hrefCom)
                        testCo.append(hrefCom)
                       
            else:
                testCo.append('NAN')
                
        testCode.append(testCo)
    return testCode

In [294]:
CompCodeArr=getCodeOfCompany(CompNameArr)

In [296]:
CompCodeArr

[[],
 [],
 ['603008'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['603317'],
 [],
 [],
 ['600560'],
 [],
 [],
 [],
 [],
 [],
 ['603976'],
 [],
 [],
 [],
 [],
 [],
 ['002800'],
 [],
 ['603068'],
 [],
 ['002911'],
 [],
 ['002891'],
 [],
 ['002368'],
 [],
 ['600105'],
 [],
 [],
 [],
 ['002297'],
 [],
 ['601880'],
 [],
 ['002677'],
 [],
 ['600593'],
 [],
 ['000333'],
 [],
 ['002347'],
 [],
 ['603001'],
 [],
 ['600321'],
 [],
 ['600891'],
 [],
 [],
 [],
 ['603299'],
 [],
 ['002272'],
 [],
 ['603398'],
 [],
 [],
 [],
 ['002379'],
 [],
 ['002515'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['000767'],
 [],
 ['600298'],
 [],
 ['002108'],
 [],
 ['600717'],
 [],
 ['600830'],
 [],
 ['000750'],
 [],
 [],
 [],
 ['600859'],
 [],
 ['002757'],
 [],
 ['002603'],
 [],
 ['600865'],
 [],
 ['600723'],
 [],
 ['000548'],
 [],
 [],
 [],
 ['002490'],
 [],
 ['600841'],
 [],
 ['002194'],
 [],
 ['002028'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['000572'],
 [],
 [],
 [],
 ['300359'],
 [],
 [],
 ['002186'],
 [],
 [],
 

In [297]:
len(CompCodeArr)

817

In [310]:
len(fnews['realTotCom'])

817

In [311]:
for index in range(len(CompCodeArr)):
    if len(CompCodeArr[index]) >1:
        del CompCodeArr[index][1] 

In [312]:
len(CompCodeArr)

817

In [315]:

def getStockInformation(testCode):
    # 用相关的公司的coed，查找股票数据信息：totstockInf

    # url='http://quotes.money.163.com/trade/lsjysj_600000.html?year=2019&season=1'

    totstockInf = []
#第一季度   code)+'.html?year=2019&season=1'
    for index in range(len(testCode)):
    # for index in range(3):
        EachstokeInf = []
        code = ''
        #     url = 'http://quotes.money.163.com/trade/lsjysj_' +str(code)+'.html?year=2019&season=1'
        if len(testCode[index]) != 0:
            code = code.join(testCode[index])
            url = 'http://quotes.money.163.com/trade/lsjysj_' + str(code) + '.html?year=2019&season=1'
            agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
            cookie = 'qgqp_b_id=de6f5b2aab0bf1b85367cff30462fbd1; st_pvi=64069861991421; st_sp=2019-01-24%2018%3A36%3A57; st_si=94918195918945; st_asi=delete; bdshare_firstime=1548326400263; st_sn=12; st_psi=2019012418445425-111000300841-9825945936'
            refer = 'http://finance.eastmoney.com/news/cgnjj_1.html'
            headers = {'User-Agent': userAgent(), 'Cookie': cookie, 'Refer': refer}

            r = requests.get(url, headers)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text, 'html.parser')
            for eachurl in soup.select('tr.dbrow'):
                hrefCom = eachurl.select('td')
                hrefCom = str(hrefCom).replace('[<td>', ' ').replace('</td>,', ' ').replace('<td class="cGreen">',
                                                                                            ' ').replace(
                    '<td class="cRed">', ' ').replace(' ', ' ').replace('<td>', ' ').replace('</td>]', ' ')
                EachstokeInf.append(hrefCom)
                # print(hrefCom)
#第二季度     .html?year=2019&season=2'             
            code = code.join(testCode[index])
            url = 'http://quotes.money.163.com/trade/lsjysj_' + str(code) + '.html?year=2019&season=2'
            agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
            cookie = 'qgqp_b_id=de6f5b2aab0bf1b85367cff30462fbd1; st_pvi=64069861991421; st_sp=2019-01-24%2018%3A36%3A57; st_si=94918195918945; st_asi=delete; bdshare_firstime=1548326400263; st_sn=12; st_psi=2019012418445425-111000300841-9825945936'
            refer = 'http://finance.eastmoney.com/news/cgnjj_1.html'
            headers = {'User-Agent': userAgent(), 'Cookie': cookie, 'Refer': refer}                
            r = requests.get(url, headers)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text, 'html.parser')                
            for eachurl in soup.select('tr.dbrow'):
                hrefCom = eachurl.select('td')
                hrefCom = str(hrefCom).replace('[<td>', ' ').replace('</td>,', ' ').replace('<td class="cGreen">',
                                                                                            ' ').replace(
                    '<td class="cRed">', ' ').replace(' ', ' ').replace('<td>', ' ').replace('</td>]', ' ')
                EachstokeInf.append(hrefCom)                
                
            
        else:
            a = 0
            EachstokeInf.append('NAN')
        totstockInf.append(EachstokeInf)

    return totstockInf


In [316]:
StockInformation=getStockInformation(CompCodeArr)

In [317]:
StockInformation

[['NAN'],
 ['NAN'],
 ['[]',
  ' 2019-03-28   10.67   10.98   10.55   10.65   -0.04   -0.37   52,103   5,597   4.02   1.63 ',
  ' 2019-03-26   11.27   11.33   10.84   10.90   -0.35   -3.11   74,325   8,231   4.36   2.33 ',
  ' 2019-03-22   11.09   11.35   10.81   11.22   0.15   1.36   109,130   12,147   4.88   3.42 ',
  ' 2019-03-20   10.80   10.80   10.45   10.65   -0.16   -1.48   71,654   7,622   3.24   2.24 ',
  ' 2019-03-18   10.40   10.65   10.29   10.61   0.24   2.31   92,279   9,713   3.47   2.89 ',
  ' 2019-03-14   11.01   11.04   10.08   10.32   -0.69   -6.27   111,149   11,631   8.72   3.48 ',
  ' 2019-03-12   11.43   11.64   11.11   11.33   -0.08   -0.70   117,149   13,305   4.65   3.67 ',
  ' 2019-03-08   11.90   12.15   11.28   11.42   -0.72   -5.93   97,893   11,439   7.17   3.06 ',
  ' 2019-03-06   10.62   11.75   10.62   11.75   1.03   9.61   150,776   16,824   10.54   4.72 ',
  ' 2019-03-04   10.16   10.36   10.06   10.23   0.16   1.59   123,643   12,653   2.98   3.87 '

In [253]:
StockInformation=StockInformation1+StockInformation2

In [254]:
StockInformation

[[],
 [],
 ['[]',
  ' 2019-03-28   10.67   10.98   10.55   10.65   -0.04   -0.37   52,103   5,597   4.02   1.63 ',
  ' 2019-03-26   11.27   11.33   10.84   10.90   -0.35   -3.11   74,325   8,231   4.36   2.33 ',
  ' 2019-03-22   11.09   11.35   10.81   11.22   0.15   1.36   109,130   12,147   4.88   3.42 ',
  ' 2019-03-20   10.80   10.80   10.45   10.65   -0.16   -1.48   71,654   7,622   3.24   2.24 ',
  ' 2019-03-18   10.40   10.65   10.29   10.61   0.24   2.31   92,279   9,713   3.47   2.89 ',
  ' 2019-03-14   11.01   11.04   10.08   10.32   -0.69   -6.27   111,149   11,631   8.72   3.48 ',
  ' 2019-03-12   11.43   11.64   11.11   11.33   -0.08   -0.70   117,149   13,305   4.65   3.67 ',
  ' 2019-03-08   11.90   12.15   11.28   11.42   -0.72   -5.93   97,893   11,439   7.17   3.06 ',
  ' 2019-03-06   10.62   11.75   10.62   11.75   1.03   9.61   150,776   16,824   10.54   4.72 ',
  ' 2019-03-04   10.16   10.36   10.06   10.23   0.16   1.59   123,643   12,653   2.98   3.87 ',
  ' 2019

In [318]:
def getStockDateOpenClose (totstockInf):

    falldate = []
    fallopenPri = []
    fallclosePri = []
    for indexT in range(len(totstockInf)):
    # for indexT in range(1):
        date = []
        openPri = []
        closePri = []
        for index in range(len(totstockInf[indexT])):
            if(len(totstockInf[indexT][index])>0):
                lines = ''
                lines = lines.join(totstockInf[indexT][index])
                lines = lines.split()
                a = 0
                for dat in lines:
                    a = a + 1
                    if (a == 1):
                        date.append(dat)
                    if (a == 2):
                        openPri.append(dat)
                    if (a == 5):
                        closePri.append(dat)
            else :
                date.append('null')
                openPri.append('null')
                closePri.append('null')
        falldate.append(date)
        fallopenPri.append(openPri)
        fallclosePri.append(closePri)
    return falldate,fallopenPri,fallclosePri

 

In [319]:
date,openPri,closePri = getStockDateOpenClose(StockInformation)

In [178]:
fnews.head()

,Unnamed: 0,title,date,source,content,realTotCom
0,0,TCL集团究竟能让半导体显示业务喷发出多大能量？,2019年03月24日 09:01,"经济观察报<span>来源：</span><ahref=""http://media.east...","[<p style=""display:none;height:1px;overflow:hi...",['集团']
1,1,从不炒股的吴晓波为何看上了“最妖股王”全通教育,2019年03月24日 00:06,"经济观察报<span>来源：</span><ahref=""http://media.east...","[<p style=""text-align:center;""><a class=""EmIma...",['全通教育']
2,2,烤鸭门店增加食客却变少 全聚德去年净利下滑近五成,2019年03月24日 01:51,"每日经济新闻<span>来源：</span><ahref=""http://media.eas...","[<p style=""text-align:center;""><a class=""EmIma...",['全聚德']
3,3,*ST椰岛卖房保壳 经销商称不敢拿货怕烫手,2019年03月24日 02:31,"华夏时报<span>来源：</span><ahref=""http://media.eastm...","[<p style=""text-align:center;""><a class=""EmIma...",['椰岛']
4,4,新增贷款六成投向民企 浙商银行全力冲刺回A,2019年03月23日 23:01,"中国经营报<span>来源：</span><ahref=""http://media.east...","[<p style=""display:none;height:1px;overflow:hi...",[]


In [320]:
def DivDate(date):
    final=[]
    for index in range(len(date)):
        haha=[]
        for indexT in range(len(date[index])):
                fenge=str(date[index][indexT]).split('-')
                haha.append(fenge)
               
        final.append(haha)
    return final

In [321]:
date=DivDate(date)

In [322]:
storkDate=date

In [204]:
#去除中文
def cleantxt(raw):
    fil = re.compile(u'[^0-9a-zA-Z ]', re.UNICODE)
    return fil.sub('-', raw)
# 将新闻日期切成单个
def DivNewsDate(date):
    NewsDate=[]
    NewsTitleDate=[]
    for news in date:
        b=''
        b=b.join(news)
        haha=cleantxt(b)
        NewsDate.append(haha)
    return NewsDate
# 将新闻日期切成单个
def getNewsDate(NewssDate):
    NewsTitleDate=[]
    for index in NewssDate:
        NewsTitleDate.append(index.split('-'))

    return NewsTitleDate

In [200]:
ttt=DivNewsDate(fnews['date'])

In [203]:
ttt

['2019-03-24- 09-01',
 '2019-03-24- 00-06',
 '2019-03-24- 01-51',
 '2019-03-24- 02-31',
 '2019-03-23- 23-01',
 '2019-03-23- 20-42',
 '2019-03-23- 19-58',
 '2019-03-23- 19-53',
 '2019-03-23- 19-34',
 '2019-03-23- 19-34',
 '2019-03-23- 19-30',
 '2019-03-23- 19-23',
 '2019-03-23- 19-00',
 '2019-03-23- 17-22',
 '2019-03-23- 17-13',
 '2019-03-23- 17-03',
 '2019-03-23- 17-03',
 '2019-03-23- 16-14',
 '2019-03-23- 14-25',
 '2019-03-23- 14-19',
 '2019-03-23- 14-18',
 '2019-03-23- 14-17',
 '2019-03-23- 14-14',
 '2019-03-23- 14-09',
 '2019-03-23- 14-06',
 '2019-03-23- 14-05',
 '2019-03-23- 13-52',
 '2019-03-23- 13-29',
 '2019-03-23- 11-13',
 '2019-03-23- 10-57',
 '2019-03-23- 10-54',
 '2019-03-23- 10-35',
 '2019-03-23- 10-26',
 '2019-03-23- 10-25',
 '2019-03-23- 10-08',
 '2019-03-23- 10-07',
 '2019-03-23- 09-49',
 '2019-03-23- 09-40',
 '2019-03-23- 09-28',
 '2019-03-23- 09-00',
 '2019-03-23- 09-00',
 '2019-03-23- 08-21',
 '2019-03-23- 07-37',
 '2019-03-23- 06-45',
 '2019-03-23- 06-39',
 '2019-03-

In [205]:
##新闻的日期['2019', '03', '20', '', '19', '56']
NewscDate=getNewsDate(ttt)

In [206]:
NewscDate

[['2019', '03', '24', ' 09', '01'],
 ['2019', '03', '24', ' 00', '06'],
 ['2019', '03', '24', ' 01', '51'],
 ['2019', '03', '24', ' 02', '31'],
 ['2019', '03', '23', ' 23', '01'],
 ['2019', '03', '23', ' 20', '42'],
 ['2019', '03', '23', ' 19', '58'],
 ['2019', '03', '23', ' 19', '53'],
 ['2019', '03', '23', ' 19', '34'],
 ['2019', '03', '23', ' 19', '34'],
 ['2019', '03', '23', ' 19', '30'],
 ['2019', '03', '23', ' 19', '23'],
 ['2019', '03', '23', ' 19', '00'],
 ['2019', '03', '23', ' 17', '22'],
 ['2019', '03', '23', ' 17', '13'],
 ['2019', '03', '23', ' 17', '03'],
 ['2019', '03', '23', ' 17', '03'],
 ['2019', '03', '23', ' 16', '14'],
 ['2019', '03', '23', ' 14', '25'],
 ['2019', '03', '23', ' 14', '19'],
 ['2019', '03', '23', ' 14', '18'],
 ['2019', '03', '23', ' 14', '17'],
 ['2019', '03', '23', ' 14', '14'],
 ['2019', '03', '23', ' 14', '09'],
 ['2019', '03', '23', ' 14', '06'],
 ['2019', '03', '23', ' 14', '05'],
 ['2019', '03', '23', ' 13', '52'],
 ['2019', '03', '23', ' 13',

In [209]:
#修改新闻时间，将下午2点45之后的新闻，参考的股票日期往下一天
def changeDateNews(NewsDate):
    for indexNews in range(len(NewsDate)):
        if(len(NewsDate[indexNews]) is 6):
            if (int(NewsDate[indexNews][4]) == 14 and int(NewsDate[indexNews][5] >= 45 )) or (int(NewsDate[indexNews][4]) > 14):
                #if the month is 1,3,5,7,8,10,12
                if(NewsDate[indexNews][1]==1 or NewsDate[indexNews][1]==3 or NewsDate[indexNews][1]==5 or NewsDate[indexNews][1]==7 or NewsDate[indexNews][1]==8 or NewsDate[indexNews][1]==10 or NewsDate[indexNews][1]==12):
                    # if the day is less than 31
                    if(NewsDate[indexNews][2]<31):
                        NewsDate[indexNews][2]=NewsDate[indexNews][2]+1
                    else:
                        if(NewsDate[indexNews][1] is not 12):
                            NewsDate[indexNews][1]=int(NewsDate[indexNews][1])+1
                        else :
                            NewsDate[indexNews][0]=int(NewsDate[indexNews][0])+1
                else:
                                    # if the day is less than 30
                    if(int(NewsDate[indexNews][2])<30):
                        NewsDate[indexNews][2]=str(int(NewsDate[indexNews][2])+1)
                    else:
                        if(int(NewsDate[indexNews][1]) is not 12):
                            NewsDate[indexNews][1]=str(int(NewsDate[indexNews][1])+1)
                        else :
                            NewsDate[indexNews][0]=str(int(NewsDate[indexNews][0])+1)
    return NewsDate


In [210]:
changedNewsDate=changeDateNews(NewscDate)

In [211]:
changedNewsDate

[['2019', '03', '24', ' 09', '01'],
 ['2019', '03', '24', ' 00', '06'],
 ['2019', '03', '24', ' 01', '51'],
 ['2019', '03', '24', ' 02', '31'],
 ['2019', '03', '23', ' 23', '01'],
 ['2019', '03', '23', ' 20', '42'],
 ['2019', '03', '23', ' 19', '58'],
 ['2019', '03', '23', ' 19', '53'],
 ['2019', '03', '23', ' 19', '34'],
 ['2019', '03', '23', ' 19', '34'],
 ['2019', '03', '23', ' 19', '30'],
 ['2019', '03', '23', ' 19', '23'],
 ['2019', '03', '23', ' 19', '00'],
 ['2019', '03', '23', ' 17', '22'],
 ['2019', '03', '23', ' 17', '13'],
 ['2019', '03', '23', ' 17', '03'],
 ['2019', '03', '23', ' 17', '03'],
 ['2019', '03', '23', ' 16', '14'],
 ['2019', '03', '23', ' 14', '25'],
 ['2019', '03', '23', ' 14', '19'],
 ['2019', '03', '23', ' 14', '18'],
 ['2019', '03', '23', ' 14', '17'],
 ['2019', '03', '23', ' 14', '14'],
 ['2019', '03', '23', ' 14', '09'],
 ['2019', '03', '23', ' 14', '06'],
 ['2019', '03', '23', ' 14', '05'],
 ['2019', '03', '23', ' 13', '52'],
 ['2019', '03', '23', ' 13',

In [323]:
storkDate

[[['NAN']],
 [['NAN']],
 [['[]'],
  ['2019', '03', '28'],
  ['2019', '03', '26'],
  ['2019', '03', '22'],
  ['2019', '03', '20'],
  ['2019', '03', '18'],
  ['2019', '03', '14'],
  ['2019', '03', '12'],
  ['2019', '03', '08'],
  ['2019', '03', '06'],
  ['2019', '03', '04'],
  ['2019', '02', '28'],
  ['2019', '02', '26'],
  ['2019', '02', '22'],
  ['2019', '02', '20'],
  ['2019', '02', '18'],
  ['2019', '02', '14'],
  ['2019', '02', '12'],
  ['2019', '02', '01'],
  ['2019', '01', '30'],
  ['2019', '01', '28'],
  ['2019', '01', '24'],
  ['2019', '01', '22'],
  ['2019', '01', '18'],
  ['2019', '01', '16'],
  ['2019', '01', '14'],
  ['2019', '01', '10'],
  ['2019', '01', '08'],
  ['2019', '01', '04'],
  ['2019', '01', '02'],
  ['[]'],
  ['2019', '04', '17'],
  ['2019', '04', '15'],
  ['2019', '04', '11'],
  ['2019', '04', '09'],
  ['2019', '04', '04'],
  ['2019', '04', '02']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['NAN']],
 [['[]'

In [ ]:
storkDate[2][1][0]

In [334]:
###将stork里面开盘index调整：  openPri,closePri
profit=[]
realNewsindex=[] ##没有删除的数据集里面的有效信息编号
for indexT in range(len(changedNewsDate)):          #indexT表示第几个公司
    for index in range(len(storkDate[indexT])):  #这个Index可以找到那个公司那一天的索引
#         if(int(changedNewsDate[indexT][0]) == True and int(storkDate[indexT][index][0]) == True and int(changedNewsDate[indexT][1]) == True and int(storkDate[indexT][index][1]) == True and int(changedNewsDate[indexT][2]) == True and int(storkDate[indexT][index][2]) == True):
            if (changedNewsDate[indexT][0] == storkDate[indexT][index][0]) and ( changedNewsDate[indexT][1] == storkDate[indexT][index][1]) and (changedNewsDate[indexT][2] == storkDate[indexT][index][2]):
                profit.append(float(closePri[indexT][index])-float(openPri[indexT][index]))
                realNewsindex.append(indexT)

In [335]:
profit

[0.26000000000000023,
 0.009999999999999787,
 -0.08999999999999986,
 0.1999999999999993,
 -0.06999999999999984,
 0.2400000000000002]

In [336]:
realNewsindex

[121, 128, 131, 133, 134, 136]